In [91]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import mysql.connector
import time

In [68]:
connection = mysql.connector.connect(host='sfdb.c38swr1uods4.us-west-2.rds.amazonaws.com',
            user = 'sf_master_db',
            passwd = 'this_is_testing',
            db = 'sandbox_db')
assets= pd.read_sql_query("select * from api_equity_security_master", connection)

In [83]:
assets = assets[assets['bnchmrk'] == 'Russell_3000']
assets

asset_id ticker                        comnam      conid      cusip  \
3       10026   JJSF        J & J SNACK FOODS CORP     270864  466032109   
5       10032   PLXS                   PLEXUS CORP     273204  729132100   
8       10051   HNGR                    HANGER INC     755007  41043F208   
12      10104   ORCL                   ORACLE CORP  242507003  68389X105   
13      10107   MSFT                MICROSOFT CORP   11462841  594918104   
...       ...    ...                           ...        ...        ...   
7858    93423    SIX  SIX FLAGS ENTERTAINMENT CORP   75103410  83001A102   
7859    93426    VPG    VISHAY PRECISION GROUP INC   76669040  92835K103   
7860    93427     FN                      FABRINET       None   G3323L10   
7862    93429   CBOE    C B O E GLOBAL MARKETS INC   76227954  12503M108   
7868    93436   TSLA                     TESLA INC  242507025  88160R101   

              isin  valid_date siccd country_code prim_exch currency  \
3     US4660321096  2020-09-30  2052         None      None     None   
5     US7291321005  2020-09-30  3670         None      None     None   
8     US41043F2083  2020-09-30  4813         None      None     None   
12    US68389X1054  2020-09-30  7372         None      None     None   
13    US5949181045  2020-09-30  7370         None      None     None   
...            ...         ...   ...          ...       ...      ...   
7858  US83001A1025  2020-09-30  7996         None      None     None   
7859  US92835K1034  2020-09-30  3676         None      None     None   
7860          None  2020-09-30  3661         None      None     None   
7862  US12503M1080  2020-09-30  6231         None      None     None   
7868  US88160R1014  2020-09-30  9999         None      None     None   

           bnchmrk  
3     Russell_3000  
5     Russell_3000  
8     Russell_3000  
12    Russell_3000  
13    Russell_3000  
...            ...  
7858  Russell_3000  
7859  Russell_3000  
7860  Russell_3000  
7862  Russell_3000  
7868  Russell_3000  

[2835 rows x 12 columns]

In [95]:
tickers = ['GOOGL','AAPL']
API_URL = "https://www.alphavantage.co/query"
API_KEY = 'VA355PKBU1CCOU9S'

end = datetime.strftime((datetime.now() - timedelta(1)), '%Y-%m-%d')
start = datetime.strftime((datetime.now() - timedelta(3)), '%Y-%m-%d')

start_time = time.time()

df = pd.DataFrame()
for index, row in assets[:200].iterrows():
    
    pram = { "function": "TIME_SERIES_DAILY_ADJUSTED", 
    "symbol": row['ticker'],
    "outputsize" : "compact",
    "datatype": "json", 
    "apikey": API_KEY }
    response = requests.get(API_URL, pram) 
    response_json = response.json() 
    
    if 'Error Message' not in response_json:
        mkdata = pd.DataFrame.from_dict(response_json['Time Series (Daily)'], orient= 'index').sort_index(axis=1)
        mkdata['caldt'] = mkdata.index
        mask = (mkdata['caldt'] > start) & (mkdata['caldt'] <= end)
        mkdata = mkdata.loc[mask]
        mkdata['ticker'] = row['ticker']
    else:
        print(row['ticker'])
        continue
    
    pram = { "function": "OVERVIEW", 
    "symbol": row['ticker'],
    "outputsize" : "compact",
    "datatype": "json", 
    "apikey": API_KEY }
    response = requests.get(API_URL, pram) 
    response_json = response.json()
    
    if bool(response_json):
        mkdata['shr'] = response_json['SharesOutstanding']
    else:
        print(row['ticker'])
        
        
    mkdata = mkdata.rename(columns={ '1. open': 'open', '2. high': 'high', '3. low': 'low', '4. close': 'close', '5. adjusted close': 'prc', '6. volume': 'vol'})
    mkdata['prc'] = mkdata['prc'].astype(float)
    mkdata['ret'] = mkdata['prc']/mkdata['prc'].shift(-1)
    
    mkdata = mkdata[mkdata['caldt']==end]
    
    mkdata = mkdata[['prc','vol','ticker','caldt','shr','ret']]
    mkdata['asset_id'] = row['asset_id']
    mkdata['shrcd'] = 0
    mkdata['siccd'] = row['siccd']
    mkdata['rf'] = 0
    
    
    df = pd.concat([df, mkdata])

end_time = time.time()
print(end_time - start_time)

259.20043087005615


In [94]:
df

prc       vol ticker       caldt         shr       ret  \
2020-12-22  153.850     94735   JJSF  2020-12-22    18955700  0.999026   
2020-12-22   76.760     85934   PLXS  2020-12-22    28824200  0.994043   
2020-12-22   21.480    114234   HNGR  2020-12-22    38109400  0.951706   
2020-12-22   65.150  17148222   ORCL  2020-12-22  3010889984  1.010391   
2020-12-22  223.940  22643397   MSFT  2020-12-22  7560500224  1.006065   
2020-12-22  147.420   1071003   TROW  2020-12-22   227548992  0.991659   
2020-12-22  207.940   2020057    HON  2020-12-22   701686016  0.987088   
2020-12-22   53.220    510404   AMRC  2020-12-22    29866100  1.041487   
2020-12-22  194.110    270686   RGEN  2020-12-22    54717300  1.005230   
2020-12-22    1.970    308129     AT  2020-12-22    89222600  1.000000   
2020-12-22   59.000    241368   BWXT  2020-12-22    95309600  0.991930   
2020-12-22   13.900     31604  PTVCB  2020-12-22    11662500  0.970670   
2020-12-22   71.590    104648   INDB  2020-12-22    32965100  0.979745   
2020-12-22   27.900    297821    HVT  2020-12-22    16699700  0.970097   
2020-12-22   67.830     28265    TMP  2020-12-22    14931100  0.977096   
2020-12-22  113.240     84461   BCPC  2020-12-22    32351100  1.003456   
2020-12-22  332.260     17218   DJCO  2020-12-22     1380750  0.996790   
2020-12-22   57.340     57098   ASTE  2020-12-22    22609300  0.987089   
2020-12-22   39.820    342061   WERN  2020-12-22    69097904  0.993513   
2020-12-22   39.600    281681   SKYW  2020-12-22    50140200  0.994225   
2020-12-22   45.010      6510   FRPH  2020-12-22     9418380  1.005810   
2020-12-22   94.350     51822   AMWD  2020-12-22    16993300  0.991905   
2020-12-22   48.730   1600118    ADM  2020-12-22   556388992  0.987037   
2020-12-22   17.130    768004    AAN  2020-12-22    33778500  0.951138   
2020-12-22   21.060    339522   VIVO  2020-12-22    43123100  0.972299   
2020-12-22   25.150     70230   CLFD  2020-12-22    13647000  0.999603   
2020-12-22  118.110    138489    WTS  2020-12-22    27433900  1.008625   
2020-12-22    9.210   1723061    FNB  2020-12-22   322635008  0.990323   
2020-12-22   82.360    378135    EHC  2020-12-22    99432496  1.002923   
2020-12-22  110.520   3802681   FISV  2020-12-22   668620032  0.982487   
2020-12-22  572.720    102926  FCNCA  2020-12-22     8811220  0.980886   
2020-12-22   18.350    138871   HTLD  2020-12-22    80798496  0.995119   
2020-12-22   11.770    724538   OSUR  2020-12-22    71732800  1.005124   
2020-12-22   13.170    479707    CAL  2020-12-22    37906800  0.994713   
2020-12-22   82.590    834152     BC  2020-12-22    78342600  1.015118   
2020-12-22   18.540    344991    UIS  2020-12-22    63037900  1.011457   
2020-12-22   42.530    327652   WSFS  2020-12-22    50402400  1.009734   
2020-12-22  156.210      5343   ITIC  2020-12-22     1892410  0.991810   
2020-12-22   77.380   1612236   CERN  2020-12-22   306590016  1.010315   
2020-12-22   41.070    351875    WBS  2020-12-22    90201200  0.979490   
2020-12-22  980.800     40159    MKL  2020-12-22    13777800  0.992210   
2020-12-22   17.670    195810   AXGN  2020-12-22    40374000  1.035149   
2020-12-22   17.300     34595   FLIC  2020-12-22    23887100  0.984633   
2020-12-22    8.680   1308907    FBP  2020-12-22   218228992  0.980791   
2020-12-22   15.420     96068    AVD  2020-12-22    30287600  1.029333   
2020-12-22   10.935    389923   PLAB  2020-12-22    65663600  0.992287   
2020-12-22  110.250     38382   CRMT  2020-12-22     6624650  0.975923   
2020-12-22   42.560     90010   ARGO  2020-12-22    34684700  0.997656   
2020-12-22   17.120    101332    OFG  2020-12-22    51347100  0.980527   
2020-12-22    8.030    176640   CATO  2020-12-22    21170400  0.972155   

           asset_id  shrcd siccd  rf  
2020-12-22    10026      0  2052   0  
2020-12-22    10032      0  3670   0  
2020-12-22    10051      0  4813   0  
2020-12-22    10104      0  7372   0  
2020-12-22    10107      0  7370   0  
2020-12

In [77]:
import pprint
pram = { "function": "OVERVIEW", 
"symbol": 'EWST',
"apikey": "VA355PKBU1CCOU9S" }
response = requests.get(API_URL, pram) 
response_json = response.json() 

print(type(response.json()))
# ovdata = pd.DataFrame.from_dict(response_json['Time Series (Daily)'], orient= 'index').sort_index(axis=1)

# ovdata

<class 'dict'>


In [81]:
pram = { "function": "TIME_SERIES_DAILY_ADJUSTED", 
"symbol": 'BTFG',
"outputsize" : "compact",
"datatype": "json", 
"apikey": API_KEY }
response = requests.get(API_URL, pram) 
response_json = response.json() 

pprint.pprint(response_json)
# mkdata = pd.DataFrame.from_dict(response_json['Time Series (Daily)'], orient= 'index').sort_index(axis=1)

# mkdata

{'Error Message': 'Invalid API call. Please retry or visit the documentation '
                  '(https://www.alphavantage.co/documentation/) for '
                  'TIME_SERIES_DAILY_ADJUSTED.'}
